# Auto Polling
    - The original persona generation worked, but the personas were too generic and didnt give a model context to 
        make accurate predictions of the beliefs of a person
    - Now making a new pipeline that will create personas in different stages to 
        

In [1]:
import Utils.Chat_GPT_Funcs as GPT
import concurrent.futures
import os
import json
import string
import random
import numpy as np
import pickle

In [2]:
def create_directory_if_not_exists(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# Load First and Last Names

In [3]:
eths = ["White", "American Indian or Alaska Native", "Black or African American", "Asian", "Hispanic or Latino"]
genders = ['Men', 'Women']

dems = []
for e in eths:
    for g in genders:
        s = f"{e} {g}"
        dems.append(s)
        
first_names = {}
for d in dems:
    # Open the file in read binary mode
    with open(f"Names/First_Names/{d}.pkl", "rb") as file:
        # Use pickle to load the list object from the file
        my_list = pickle.load(file)
        first_names[d] = my_list

last_names = {}
for e in eths:
    # Open the file in read binary mode
    with open(f"Names/Last_Names/{e}.pkl", "rb") as file:
        # Use pickle to load the list object from the file
        my_list = pickle.load(file)
        last_names[e] = my_list

# Top 44 Universities in USA

In [4]:
universities = [
    "Massachusetts Institute of Technology (MIT)",
    "Stanford University",
    "Harvard University",
    "California Institute of Technology (Caltech)",
    "University of Chicago",
    "University of Pennsylvania",
    "Princeton University",
    "Yale University",
    "Cornell University",
    "Columbia University",
    "Johns Hopkins University",
    "University of Michigan-Ann Arbor",
    "University of California Berkeley (UCB)",
    "Northwestern University",
    "New York University (NYU)",
    "University of California, Los Angeles (UCLA)",
    "Duke University",
    "Carnegie Mellon University",
    "University of California, San Diego (UCSD)",
    "Brown University",
    "University of Texas at Austin",
    "University of Washington",
    "University of Wisconsin-Madison",
    "University of Illinois at Urbana-Champaign",
    "Georgia Institute of Technology (Georgia Tech)",
    "Pennsylvania State University",
    "Rice University",
    "University of California, Davis (UCD)",
    "University of North Carolina, Chapel Hill",
    "Boston University",
    "Michigan State University",
    "Texas A&M University",
    "University of Maryland, College Park",
    "Case Western Reserve University",
    "University of Pittsburgh",
    "University of Minnesota, Twin Cities",
    "University of Florida",
    "Vanderbilt University",
    "Dartmouth College",
    "Arizona State University",
    "University of California, Irvine (UCI)",
    "University of Notre Dame",
    "Yeshiva University",
    "University of Massachusetts, Amherst"
]

# Popular Majors in the USA

In [5]:
majors = [
    "Business Administration",
    "Accounting",
    "Nursing",
    "Psychology",
    "Communications",
    "Marketing",
    "Education",
    "Elementary Education",
    "English",
    "Computer Science",
    "Finance",
    "Criminal Justice",
    "Biology",
    "Political Science",
    "Economics",
    "Electrical Engineering",
    "History",
    "Liberal Arts",
    "Sociology",
    "Fine Arts",
    "Commercial Art & Graphic Design",
    "General Engineering",
    "Journalism",
    "Computer and Information Systems",
    "Social Work"
]

# Distributing Statistics

In [6]:
def distribute_university():
    # For now, every university has the same chance of being selected
    # Assuming 'universities' is your list
    random_university = random.choice(universities)
    
    return random_university
    
def distribute_gender():
    random_number = random.random()
    
    if random_number < 0.49:
        return "Female"
    else:
        return "Male"
    
def distribute_ethnicity():
    random_number = random.random()
    
    if random_number < 0.52:
        return "White"
    elif random_number < 0.55:
        return "American Indian or Alaska Native"
    elif random_number < 0.73:
        return "Black or African American"
    elif random_number < 0.8:
        return "Asian"
    else:
        return "Hispanic or Latino"
    
def distribute_degree_level_and_age():
    """
    Current distribution is not real, just assumed based on probable breakdown
    
    Then after that, pull a random age that they would like be in that program
    """
    random_number = random.random()
    
    if random_number < 0.60:
        return "Bachelor's Degree", random.randint(18, 22)
    elif random_number < 0.94:
        return "Master's Degree", random.randint(22, 30)
    else:
        return "PHD", random.randint(26, 35)
    
def distribute_major():
    """
    Currently all majors have the same probability
    """
    random_major = random.choice(majors)
    
    return random_major

# Generate Names

In [7]:
def generate_name(eth, gend):
    if gend == 'Male':
        gen = 'Men'
    else:
        gen = 'Women'
    f_names = first_names[f"{eth} {gen}"]
    
    first = f_names[random.randint(0,len(f_names)-1)].strip()
    
    l_names = last_names[f"{eth}"]
    last = l_names[random.randint(0,len(l_names)-1)].strip()
    
    return f"{first} {last}"

In [8]:
def make_persona():
    p = {}
    
    # Distribute Ethnicity
    eth = distribute_ethnicity()
    # Distribute Gender
    gend = distribute_gender()
    
    p['Name'] = generate_name(eth, gend)
    
    # Distribute Age and Income
    degree, age = distribute_degree_level_and_age()
    p['Age'] = str(age)
    p['Ethnicity'] = eth
    p['Gender'] = gend
    
    p['Degree'] = degree

    # Distribute Education Level
    p['Major'] = distribute_major()
    
    p['University'] = distribute_university()
    
    # Initialize a list of tuples that will be the question answers
    p['QA'] = []
    
    return p

In [9]:
poll_size = 10
personas = []
for i in range(poll_size):
    personas.append(make_persona())
print(personas[0])

{'Name': 'Jazlyn Phillips', 'Age': '29', 'Ethnicity': 'Black or African American', 'Gender': 'Female', 'Degree': "Master's Degree", 'Major': 'Accounting', 'University': 'University of Washington', 'QA': []}


In [10]:
def persona_to_string(persona):

    return (f"Imagine you are {persona['Name']}, a {persona['Age']} year old {persona['Ethnicity']} {persona['Gender']} student at {persona['University']}. "
            f"You are pursuing a {persona['Degree']} in {persona['Major']}. "
           )

In [11]:
questions = [
    "How would your perfect professor provide constructive feedback on your assignments and projects?",
    "What is your best thing a professor has done to aid in your learning during your college career?"
]

In [12]:
def ask_student(p, question):
    
    student_role = persona_to_string(p)
    
    first_name = p['Name'].split(" ")[0]
    
    q_in = f"{first_name}, could you please concisely answer the following question: {question}"
    
    answer, usage = GPT.chat_gpt(q_in, engine='gpt-3.5-turbo-0613', temp=0.5, tokens = 600, role=student_role)
    
    return answer, usage

In [13]:
def ask_student_questions(p, questions, num_follow_up=1, retries=3):  
    global total_prompt_tokens, total_completion_tokens
    try: 
        for question in questions:
            # Ask the question to the persona
            attempt = 0
            while attempt <= retries:
                try:   
                    answer, usage = ask_student(p, question)

                    # Add Usage to the totals for bookkeeping
                    total_completion_tokens += usage['completion_tokens']
                    total_prompt_tokens += usage['prompt_tokens']
                    
                    # Append the question answer to the personas list
                    p['QA'].append((question, answer))
                    
                    break
                    
                except Exception as e:
                    attempt += 1
                    print(f"Attempt {attempt} on date {day.date} - Error while generating themes: {e}")
                    if attempt > retries:
                        print("Failed to generate themes after several attempts. Exiting.")
                        return None
                    time.sleep(1)  # Wait for 1 second before retrying, this could be adjusted
    except Exception as e:
        # If any error occurs that has not been handled, return the exception
        print(f"Error on date: {day.date} with exception: {e}")
        return e
    
    
    try:
        # Create and ask follow up questions
        for i in range(num_follow_up):
            # Ask the question to the persona
            attempt = 0
            while attempt <= retries:
                try:   
                    print("teststs")
                    question_gen_role = GPT.open_file("Prompts/follow_up_gen_role.txt")
                    
                    question_gen_in = "Please create a follow up question from the following:\n\n"
                    
                    # Get the last 2 questions asked
                    qa_len = len(p['QA'])
                    q1 = p['QA'][qa_len-2]
                    q2 = p['QA'][qa_len-1]
                    
                    # Add the first question
                    question_gen_in += f"Question: {q1[0]}\n"
                    question_gen_in += f"Answer: {q1[1]}\n\n"
                    
                    # Add the second question
                    question_gen_in += f"Question: {q2[0]}\n"
                    question_gen_in += f"Answer: {q2[1]}\n\n"
                    
                    new_q_text, usage = GPT.chat_gpt(question_gen_in, engine = 'gpt-3.5-turbo-0613' , role = question_gen_role, temp = 0.45, tokens = 1200)
                    
                    try:
                        new_question = json.loads(new_q_text)['Question']
                    except:
                        print("the fuck")
                    
                    new_answer, usage = ask_student(p, new_question)

                    # Add Usage to the totals for bookkeeping
                    total_completion_tokens += usage['completion_tokens']
                    total_prompt_tokens += usage['prompt_tokens']
                    
                    # Append the question answer to the personas list
                    p['QA'].append((new_question, new_answer))
                    
                    break
                    
                except Exception as e:
                    attempt += 1
                    print(f"Attempt {attempt} on date {day.date} - Error while generating themes: {e}")
                    if attempt > retries:
                        print("Failed to generate themes after several attempts. Exiting.")
                        return None
                    time.sleep(1)  # Wait for 1 second before retrying, this could be adjusted
        
        print("Success")
        return "Success"
        
    except Exception as e:
        # If any error occurs that has not been handled, return the exception
        print(f"Error on date: {day.date} with exception: {e}")
        return e

In [14]:
total_completion_tokens = 0
total_prompt_tokens = 0

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks to the executor for each persona
    futures = [executor.submit(ask_student_questions, persona, questions) for persona in personas]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

teststs
teststs
teststs
teststs
teststs
teststs
teststs
Success
teststs
Success
the fuck
Success
teststs
Success
Success
the fuck
Success
Success
Error communicating with OpenAI: The server is overloaded or not ready yet.
teststs
Success
Total Prompt Tokens: 2089
Total Completion Tokens: 3011

Token Cost = $ 0.010199999999999999


In [17]:
for qa in personas:
    print(len(qa['QA']))
    print()

3

3

3

3

3

2

3

2

3

3



In [18]:
import json
file_path = "unversitai_personas_with_QA.json"

# Open the file in write mode and save the dictionary as JSON
with open(file_path, "w") as json_file:
    json.dump(personas, json_file, indent=4)